# Acceso programático a Uniprot

Uniprot también ofrece una REST-API para acceder a sus datos.

Esta interfaz es bastante mas sencilla que la de NCBI.

In [ ]:
library(httr)

In [ ]:
base_url   <- "https://rest.uniprot.org/uniprotkb/search"

Al igual que NCBI, podemos hacer búsquedas por texto, especificando campos lo hacíamos en NCBI, aunque con un formato diferente.

Para escribir nuestra búsqueda tenemos que conocer los campos disponibles:

Estos están en: https://www.uniprot.org/help/query-fields.

El texto de búsqueda tiene que tener una cierta sintaxis: https://www.uniprot.org/help/text-search.

Además tenemos que buscar definir el formato en el que queremos los datos: Tipicamente, en formato tabular y fasta para las secuencias.

Por último que tenemos que definir que datos queremos recuperar: https://www.uniprot.org/help/return_fields

Por ejemplo: queremos recuperar datos de proteínas polymerases de E. coli que

pertenezcan a Swiss-Prot (es decir que estén curadas manualmente).

In [ ]:
query <- 'protein_name:(RNA polymerase) AND (organism_name:"Escherichia coli") AND (reviewed:true)'

respuesta <- GET(
    base_url,
    query=list(query=query, format="tsv", fields="id,accession,protein_name,go_id")
)


In [ ]:
read.table(text=content(respuesta), header=TRUE, sep="\t")

In [ ]:
ecoli_pol <- read.table(text=content(respuesta), header=TRUE, sep="\t")
head(ecoli_pol)

Podemos descargar los secuencias de estas proteinas en formato fasta
cambiando el formato de la respuesta.

[Uniprot reference](https://www.uniprot.org/help/api_queries#list-of-all-formats)

In [ ]:
query <- 'protein_name:(RNA polymerase) AND (organism_name:"Escherichia coli") AND (reviewed:true)'

respuesta <- GET(
    base_url,
    query=list(
        query=query,
        format="fasta",
        fields="id,protein_name"
    )
)


In [ ]:
?httr

In [ ]:
status_code(respuesta)

In [ ]:
# Podemos recuperar las secuencias de los registros de a uno
query <- paste(paste("accession:", ecoli_pol$Entry[1]), collapse=" ")
respuesta <- GET(
    base_url,
    query=list(query=query, format="fasta")
)


In [ ]:
cat(content(respuesta))

Como acceder a las secuencias

In [ ]:
# Para leer formato fasta
# install.packages('seqinr')
# library(seqinr)
sequences <- read.fasta(file=textConnection(content(respuesta, as="text")))

# sequences[[1]]

# summary(sequences[[1]])

# attributes(sequences[[1]])

# attributes(sequences[[1]])

## Servicio IDMapping

Uniprot tiene un servicio que permite mapear identificadores entre diferentes bases de datos.

Es necesario pasarle un lista de identificadores, y decir cuales son las bases

de datos (o tipo de identificadores) de origen y destino.

Lista de los tipos de identificadores que se pueden usar están en:

- https://rest.uniprot.org/configure/idmapping/fields

In [ ]:
library(httr)

In [ ]:
url <- "https://rest.uniprot.org/idmapping/run"

# Define the parameters
params <- list(
  ids="P21802,P00765",
  from="UniProtKB_AC-ID",
  to="UniRef90"
)

# Perform the POST request
response <- POST(
  url ,
  body = params,
  encode="multipart"
)

In [ ]:
response

### Una forma alternativa

In [1]:
library(httr2)

In [23]:
url <- "https://rest.uniprot.org/idmapping/run"

req <- request(url)

resp <- req %>% req_body_multipart(
  ids="P21802,P00765",
  from="UniProtKB_AC-ID",
  to="PDB"
) %>%
  req_perform()

In [24]:
jobid <- resp_body_json(resp)

jobid[[1]]

[1] "69af484eefefea083fbf7403e2520ac272127510"

In [25]:
url <- "https://rest.uniprot.org/idmapping/status/"
req2 <- request(
  paste0(url, jobid[[1]])
) %>%
  req_perform() %>%
  resp_body_json()

In [30]:
req2

$results
$results[[1]]
$results[[1]]$from
[1] "P21802"

$results[[1]]$to
[1] "1DJS"


$results[[2]]
$results[[2]]$from
[1] "P21802"

$results[[2]]$to
[1] "1E0O"


$results[[3]]
$results[[3]]$from
[1] "P21802"

$results[[3]]$to
[1] "1EV2"


$results[[4]]
$results[[4]]$from
[1] "P21802"

$results[[4]]$to
[1] "1GJO"


$results[[5]]
$results[[5]]$from
[1] "P21802"

$results[[5]]$to
[1] "1II4"


$results[[6]]
$results[[6]]$from
[1] "P21802"

$results[[6]]$to
[1] "1IIL"


$results[[7]]
$results[[7]]$from
[1] "P21802"

$results[[7]]$to
[1] "1NUN"


$results[[8]]
$results[[8]]$from
[1] "P21802"

$results[[8]]$to
[1] "1OEC"


$results[[9]]
$results[[9]]$from
[1] "P21802"

$results[[9]]$to
[1] "1WVZ"


$results[[10]]
$results[[10]]$from
[1] "P21802"

$results[[10]]$to
[1] "2FDB"


$results[[11]]
$results[[11]]$from
[1] "P21802"

$results[[11]]$to
[1] "2PSQ"


$results[[12]]
$results[[12]]$from
[1] "P21802"

$results[[12]]$to
[1] "2PVF"


$results[[13]]
$results[[13]]$from
[1] "P21802"

$results[[13]]$to
[1] "2PVY"


$results[[14]]
$results[[14]]$from
[1] "P21802"

$results[[14]]$to
[1] "2PWL"


$results[[15]]
$results[[15]]$from
[1] "P21802"

$results[[15]]$to
[1] "2PY3"


$results[[16]]
$results[[16]]$from
[1] "P21802"

$results[[16]]$to
[1] "2PZ5"


$results[[17]]
$results[[17]]$from
[1] "P21802"

$results[[17]]$to
[1] "2PZP"


$results[[18]]
$results[[18]]$from
[1] "P21802"

$results[[18]]$to
[1] "2PZR"


$results[[19]]
$results[[19]]$from
[1] "P21802"

$results[[19]]$to
[1] "2Q0B"


$results[[20]]
$results[[20]]$from
[1] "P21802"

$results[[20]]$to
[1] "3B2T"


$results[[21]]
$results[[21]]$from
[1] "P21802"

$results[[21]]$to
[1] "3CAF"


$results[[22]]
$results[[22]]$from
[1] "P21802"

$results[[22]]$to
[1] "3CLY"


$results[[23]]
$results[[23]]$from
[1] "P21802"

$results[[23]]$to
[1] "3CU1"


$results[[24]]
$results[[24]]$from
[1] "P21802"

$results[[24]]$to
[1] "3DAR"


$results[[25]]
$results[[25]]$from
[1] "P21802"

$results[[25]]$to
[1] "3EUU"



$failedIds
$failedIds[[1]]
[1] "P00765"

In [38]:
mapping <- as.data.frame(t(sapply(req2$results, unlist)))

In [ ]:
mapping

## Paquetes de R para conectarse a Uniprot

Hay dos paquetes de R que permiten conectarse a Uniprot:

- Uniprot.ws: https://www.bioconductor.org/packages/release/bioc/html/UniProt.ws.html
- UniprotR: https://cran.r-project.org/web/packages/UniprotR/index.html

No tienen una funcionalidad muy completa, por ejemplo carecen de la capacidad de hacer búsquedas de texto.

Aunque permiten descargar datos si se tienen los identificadores de los registros.